In [8]:
import numpy as np
import pandas as pd
import rasterio
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [9]:
block_size = 50

In [10]:
dataset = rasterio.open("./data/crs.tiff")
img_data = dataset.read(1)

In [11]:
def getImageData(lat, lon):
    x, y = dataset.index(lon, lat)
    img = img_data[x - block_size // 2:x + block_size // 2, y - block_size // 2:y + block_size // 2]

    return img

# data = pd.read_csv('training/data.csv')
# dummies = pd.get_dummies(data.label)
# data_full = pd.concat([data,dummies],axis=1)
# data_full = data_full.drop(['label'],axis=1)
# data_full['block'] = data_full.apply(lambda s: getImageData(s['lat'], s['lon']),axis=1)
# data_full.head()

data = pd.read_csv('training/data_bin.csv')
data['block'] = data.apply(lambda s: getImageData(s['lat'], s['lon']),axis=1)

y = data.windmill.values
X = data.block.values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train = np.stack(X_train)
X_test = np.stack(X_test)

# print(y_test[3])
# plt.imshow(X_test[3])
# plt.show()

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(50, 50)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

#print(model.summary())

In [12]:
model.fit(X_train, y_train, validation_split=0.1, epochs=30, verbose=1)


Epoch 1/30
3/3 [==============================] - 0s 115ms/step - loss: 358.8729 - accuracy: 0.5694 - val_loss: 129.1338 - val_accuracy: 0.6250
Epoch 2/30
3/3 [==============================] - 0s 30ms/step - loss: 169.1636 - accuracy: 0.5139 - val_loss: 53.8257 - val_accuracy: 0.6250
Epoch 3/30
3/3 [==============================] - 0s 52ms/step - loss: 202.7121 - accuracy: 0.5556 - val_loss: 186.7675 - val_accuracy: 0.6250
Epoch 4/30
3/3 [==============================] - 0s 22ms/step - loss: 157.3899 - accuracy: 0.6250 - val_loss: 2.5400 - val_accuracy: 0.8750
Epoch 5/30
3/3 [==============================] - 0s 29ms/step - loss: 99.9384 - accuracy: 0.5278 - val_loss: 48.6274 - val_accuracy: 0.6250
Epoch 6/30
3/3 [======================

In [13]:
predictions = model.predict(X_test)

rounded_predictions = np.argmax(predictions, axis=1)
print(rounded_predictions)
print(y_test);
score = model.evaluate(X_test, y_test, verbose=2)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0]
[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1]
2/2 - 0s - loss: 0.6492 - accuracy: 0.9143
Test loss: 0.6492413878440857 / Test accuracy: 0.9142857193946838


In [14]:
from sklearn.metrics import confusion_matrix
import itertools
#cm = confusion_matrix(y_true=y_test, y_pred=predictions)